In [2]:
import pandas as pd
import numpy as np

df = pd.read_excel("acciones_taller3.xlsx")
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date")

In [3]:
# Calculos de rendimientos mensuales (uno por cada mes)

# --- Rendimiento simple ------
# (P_t - P_{t-1}) / P_{t-1}
rent_mensual = df.pct_change().dropna()
# --- Rendimiento logarítmico (%) ---
# ln(P_t / P_{t-1}) * 100
rent_log = np.log(df / df.shift(1)).dropna()
# los siguientes resultados seran usando rendimientos logaritmicos para mas exactitud

In [4]:
# 1️. Promedio de rentabilidad mensual
promedios = rent_log.mean()*100

In [5]:
varianzas = rent_log.var()

# Desviación estándar de cada acción
desviaciones = rent_log.std()
stats_df = pd.DataFrame({
    "Promedio Mensual": promedios,
    "Varianza": varianzas,
    "Desviación Estándar": desviaciones 
})

In [6]:
# 3️ Matriz varianza–covarianzas y de correlaciones
cov_matrix = rent_log.cov()
corr_matrix = rent_log.corr()

In [10]:
with pd.ExcelWriter("Punto 1-3.xlsx", engine="xlsxwriter") as writer:

    workbook = writer.book

    # ------- FORMATO PREMIUM -------
    formato_titulo = workbook.add_format({
        "bold": True, "font_size": 16, "align": "center"
    })

    formato_encabezado = workbook.add_format({
        "bold": True,
        "bg_color": "#1F4E79",
        "font_color": "white",
        "border": 1,
        "align": "center"
    })

    formato_fecha = workbook.add_format({
        "border": 1,
        "align": "center",
        "num_format": "yyyy-mm-dd"
    })

    formato_porcentaje = workbook.add_format({
        "border": 1,
        "align": "center",
        "num_format": "0.00000000000%"
    })

    formato_decimal = workbook.add_format({
        "border": 1,
        "align": "center"
    })

    def escribir_tabla_ordenada(df_data, hoja, titulo, tipo="decimal"):
        """
        tipo:
            - "fecha": escribe fechas
            - "porcentaje": para rentabilidades
            - "decimal": matrices o estadísticas
        """

        # Crear hoja manualmente SIN to_excel()
        ws = workbook.add_worksheet(hoja)
        writer.sheets[hoja] = ws

        # ------------ TÍTULO ------------
        ws.merge_range(0, 0, 0, len(df_data.columns), titulo, formato_titulo)

        # ------------ ENCABEZADOS ------------
        if isinstance(df_data.index[0], pd.Timestamp):
            encabezado_primera_col = "Fecha"
        else:
            encabezado_primera_col = "Acción"

        ws.write(2, 0, encabezado_primera_col, formato_encabezado)

        for col_num, col_name in enumerate(df_data.columns, start=1):
            ws.write(2, col_num, col_name, formato_encabezado)

        # ------------ ESCRITURA DE DATOS ------------
        for i in range(df_data.shape[0]):
            indice = df_data.index[i]

            # Índice
            if isinstance(indice, pd.Timestamp):
                ws.write_datetime(i + 3, 0, indice, formato_fecha)
            else:
                ws.write(i + 3, 0, indice, formato_decimal)

            # Valores
            for j in range(df_data.shape[1]):
                valor = df_data.iloc[i, j]
                if tipo == "porcentaje":
                    ws.write(i + 3, j + 1, valor, formato_porcentaje)
                else:
                    ws.write(i + 3, j + 1, valor, formato_decimal)

        # Ajuste de ancho uniforme
        ws.set_column(0, len(df_data.columns), 20)

    # ---------------------------------------------------
    # ESCRITURA DE TODAS LAS HOJAS
    # ---------------------------------------------------
    escribir_tabla_ordenada(df, "Precios", "PRECIOS MENSUALES")

    escribir_tabla_ordenada(
        rent_mensual / 100,
        "Rend Simple (%)",
        "RENDIMIENTO SIMPLE (%)",
        tipo="porcentaje"
    )

    escribir_tabla_ordenada(
        rent_log / 100,
        "Rend Log (%)",
        "RENDIMIENTO LOGARÍTMICO (%)",
        tipo="porcentaje"
    )

    escribir_tabla_ordenada(
        stats_df,
        "Estadísticas Simple",
        "ESTADÍSTICAS DESCRIPTIVAS",
        tipo="decimal"
    )

    escribir_tabla_ordenada(
        cov_matrix,
        "Covarianza",
        "MATRIZ DE VARIANZA - COVARIANZA",
        tipo="decimal"
    )

    escribir_tabla_ordenada(
        corr_matrix,
        "Correlación",
        "MATRIZ DE CORRELACIÓN",
        tipo="decimal"
    )

print("Archivo creado correctamente: resultados_portafolio_premium.xlsx")

Archivo creado correctamente: resultados_portafolio_premium.xlsx
